In [100]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from modules.tfidf_custom import TfIdfCustom
from modules.tfidf import TfIdf
from modules.word_stats import WordStats
from modules.similarity import Similarity
from models.knnreg import KnnReg
from modules.data import fetch_fresh, load_pickle, save_pickle
from modules.stats import score_distribution_plot, words_count_plot, tfidf_custom_score_plot, topic_similarity_score_plot
import pandas as pd
# from scipy.stats import binned_statistic
import sys
import argparse
pickle_df = load_pickle("okt-5-10728c.p")
display(pickle_df)

,body,subreddit,submission,tfidf_score,tfidf_custom_score,length,sentences,common_word,words_count,stop_words_count,bad_words_count,bad_words,letter_count,emoticons,aspectual_class,reichenbach_tense,topic_similarity,label
0,i guess he did outlive his life expectancy by ...,science,Physicist Stephen Hawking dies aged 76,0,0,0,0,0,0,0,0,,0,False,0,0,0,8256
1,Woah. One of those deaths you always saw comin...,science,Physicist Stephen Hawking dies aged 76,0,0,0,0,0,0,0,0,,0,False,0,0,0,9470
2,“We have this one life to appreciate the grand...,science,Physicist Stephen Hawking dies aged 76,0,0,0,0,0,0,0,0,,0,False,0,0,0,12774
3,“I have lived with the prospect of an early de...,science,Physicist Stephen Hawking dies aged 76,0,0,0,0,0,0,0,0,,0,False,0,0,0,27266
4,Remember to look up at the stars and not down ...,science,Physicist Stephen Hawking dies aged 76,0,0,0,0,0,0,0,0,,0,False,0,0,0,9766
5,An absolutely monumental mind. The world mourn...,science,Physicist Stephen Hawking dies aged 76,0,0,0,0,0,0,0,0,,0,False,0,0,0,4173
6,A sad day for scientists and science enthusias...,science,Physicist Stephen Hawking dies aged 76,0,0,0,0,0,0,0,0,,0,False,0,0,0,12466
7,"Very sad. Let's celebrate his life though, he ...",science,Physicist Stephen Hawking dies aged 76,0,0,0,0,0,0,0,0,,0,False,0,0,0,4184
8,[removed],science,Physicist Stephen Hawking dies aged 76,0,0,0,0,0,0,0,0,,0,False,0,0,0,2666
9,"pour one out for the OG wheelin and dealin, ru...",science,Physicist Stephen Hawking dies aged 76,0,0,0,0,0,0,0,0,,0,False,0,0,0,3717


In [101]:
pd.options.display.min_rows = 100
#pd.options.display.max_rows = set larger than dataset
pd.options.display.max_columns = 20
%matplotlib inline
mpl.rcParams["figure.figsize"] = 15,8

In [102]:
df = pickle_df.copy()

In [103]:
# remove noise
df = df[df["body"] != "[removed]"]
df = df.reset_index()


In [104]:

similarity = Similarity()
similarity.score(df) # "topic_similarity"

word_stats = WordStats()
df = word_stats.score(df) # "words_count", "stop_words_count", "bad_words_count", "bad_words"

tfidf_custom = TfIdfCustom()
df = tfidf_custom.score(df) # "tfidf_custom_score"

tfidf = TfIdf()
df = tfidf.score(df) # "tfidf_score"

In [105]:

inspect = df.loc[[1201]]
print(inspect[["submission"]].to_string())
print(inspect[["body"]].to_string())
display(inspect[["topic_similarity", "tfidf_score", "words_count", "stop_words_count", "bad_words_count", "bad_words", "label"]])

idxr = 1200
display(df[["body", "submission", "topic_similarity", "tfidf_score", "words_count", "stop_words_count", "bad_words_count", "bad_words", "label"]][idxr:idxr+16])

                                                                                                                                           submission
1201  Science Ama Series: I am Stephen Hawking, theoretical physicist. Join me to talk about making the future of technology more human, reddit. AMA!
                                                                                                                                                                                                                body
1201  Good morning/afternoon professor Hawking, I always wondered, what was the one scientific discovery that has absolutely  baffled you? Recent or not. Thanks in advance if you get to this.\n\nEdit: spelling <3


,topic_similarity,tfidf_score,words_count,stop_words_count,bad_words_count,bad_words,label
1201,0.008336,4.275318,28.0,11.0,0.0,,1545.0


,body,submission,topic_similarity,tfidf_score,words_count,stop_words_count,bad_words_count,bad_words,label
1200,"Professor Hawking, thank you for doing an AMA....","Science Ama Series: I am Stephen Hawking, theo...",0.024223,7.347534,99.0,59.0,0.0,,802.0
1201,"Good morning/afternoon professor Hawking, I al...","Science Ama Series: I am Stephen Hawking, theo...",0.008336,4.275318,28.0,11.0,0.0,,1545.0
1202,"Professor Hawking, \n\nWhile many experts in t...","Science Ama Series: I am Stephen Hawking, theo...",0.003233,7.553637,96.0,54.0,0.0,,3157.0
1203,"Dear Professor Hawking,\n\nIf you were 24 or 2...","Science Ama Series: I am Stephen Hawking, theo...",0.009422,4.151175,23.0,13.0,0.0,,2072.0
1204,"Hello Professor Hawking,\n\nIf we discovered a...","Science Ama Series: I am Stephen Hawking, theo...",0.009584,3.625367,19.0,16.0,0.0,,5140.0
1205,How long will it take to get answers ?,"Science Ama Series: I am Stephen Hawking, theo...",0.000000,2.095791,5.0,4.0,0.0,,50.0
1206,"Hello Professor Hawking, thank you for doing t...","Science Ama Series: I am Stephen Hawking, theo...",0.066364,5.035748,41.0,27.0,0.0,,2285.0
1207,[deleted],"Science Ama Series: I am Stephen Hawking, theo...",0.000000,1.720045,3.0,0.0,0.0,,708.0
1208,This was a question proposed by one of my stud...,"Science Ama Series: I am Stephen Hawking, theo...",0.040634,7.382726,113.0,74.0,0.0,,2146.0
1209,"**How important do you think [simulating] ""emo...","Science Ama Series: I am Stephen Hawking, theo...",0.011513,6.791463,104.0,40.0,0.0,,1546.0


In [109]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.preprocessing import MinMaxScaler, KBinsDiscretizer
from sklearn.model_selection import train_test_split
from sklearn import metrics

feature_list = ["topic_similarity", "tfidf_score", "words_count", "stop_words_count", "bad_words_count"]

#display(data)
#y = df[["label"]]
#X = df[feature_list]

# Bin all of y first
bins = 15
est = KBinsDiscretizer(n_bins=bins, encode='ordinal', strategy='uniform')
est.fit(y)
yt = est.transform(y)
df["est_label"] = yt
#display(df[[*feature_list, "label", "est_label"]])
display(df["est_label"].value_counts())



#data = scaler.transform(data)
X_train, X_validate, y_train, y_validate = train_test_split(X, yt, test_size=0.7) # Bad idea to do random split? Split test/train into equal yt-bins

scaler = MinMaxScaler()
scaler.fit(X_train, y_train) # Don't cheat: fit only training data. y is ignored, but needs same shape[rows]

#Create a Gaussian Classifier
model = MultinomialNB()

# Train the model using the training sets
model.fit(scaler.transform(X_train), y_train)
#
y_pred = model.predict(X_validate)
#
accuracy = metrics.accuracy_score(y_validate, y_pred, normalize=True)
#
print(accuracy)




0.0     8601
1.0      194
2.0      136
3.0       74
4.0       56
5.0       41
6.0       22
7.0       19
8.0        8
10.0       3
11.0       2
14.0       2
9.0        1
13.0       1
12.0       1
Name: est_label, dtype: int64

0.08903789178231716
